<a href="https://colab.research.google.com/github/danielsoy/ADer/blob/main/colab_retrainable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# -*- coding: utf-8 -*-


In [22]:
!git clone https://github.com/OpenAOI/anodet.git

fatal: destination path 'anodet' already exists and is not an empty directory.


In [23]:
cd /content/anodet

/content/anodet


In [24]:
pip install -r requirements.txt

In [25]:
import os
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision import transforms
from tqdm import tqdm
#import anodet
from torch.optim.lr_scheduler import MultiStepLR

In [26]:
cd /content/anodet/anodet

/content/anodet/anodet


In [27]:
import anodet

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
local=False # train model with local dataset if true. train from colab mydrive if not.

In [30]:
if local:
    # Setup local paths
    DATASET_PATH = r"C:\anodet\notebooks\video_pro\working folder\indexed4augmet\datasets\DATASET A"
    MODEL_SAVE_PATH = "./trained_backbone.pth"
else:
    # Setup mydrive colab paths
    DATASET_PATH    = os.path.realpath("/content/drive/MyDrive/DATASET A")
    MODEL_SAVE_PATH = os.path.realpath("/content/drive/MyDrive/DATASET A MODEL")

In [31]:
IMAGE_SIZE = 224  # Original ResNet training size
BATCH_SIZE = 16  # Fixed batch size

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda



Setup device
<br>
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')<br>
print(f"Using device: {device}")<br>
# /content/anodet/anodet/datasets/dataset.py<br>
ustom dataset with resizing


In [33]:
cd /content/anodet

/content/anodet


rom anodet import anodet

rom anodet.anodet import datasets

rom anodet.anodet.datasets import dataset

In [34]:
from anodet.anodet.datasets.dataset import AnodetDataset
#    folder floder floder   .py script     clase.

In [35]:
class ResNetDataset(AnodetDataset):
    def __init__(self, root_dir):
        super().__init__(root_dir)
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),  # Standard ResNet size
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],  # ImageNet normalization
                std=[0.229, 0.224, 0.225]
            )
        ])
    def __getitem__(self, index):
        img, label, mask = super().__getitem__(index)
        if isinstance(img, np.ndarray):
            img = self.transform(img)
        return img, label, mask

In [36]:
def calculate_feature_loss(features):
    loss = 0
    weights = [0.1, 0.2, 0.3, 0.4]
    for feat, weight in zip(features, weights):
        b, c, h, w = feat.shape
        feat_reshaped = feat.view(b, c, -1)
        feat_norm = feat_reshaped / (torch.norm(feat_reshaped, dim=2, keepdim=True) + 1e-10)
        similarity = torch.bmm(feat_norm, feat_norm.transpose(1, 2))
        target = torch.eye(c, device=device).unsqueeze(0).expand(b, -1, -1)
        loss += weight * torch.mean((similarity - target) ** 2)
    return loss

In [37]:
def train_resnet_backbone(
    dataloader: DataLoader,
    num_epochs: int = 200,  # Original ResNet epochs
    initial_lr: float = 0.075,  # Original ResNet learning rate
    continue_training: bool = False  # New parameter
):
    model = resnet18(weights=None)

    # Load previous model if continuing training
    if continue_training:
        try:
            # Try to load the best model first
            model.load_state_dict(torch.load(MODEL_SAVE_PATH + '.best'))
            print(f"Loaded best model from {MODEL_SAVE_PATH}.best")
        except:
            # Fall back to the last saved model
            model.load_state_dict(torch.load(MODEL_SAVE_PATH))
            print(f"Loaded last saved model from {MODEL_SAVE_PATH}")

    model = model.to(device)

    # Original ResNet optimizer settings
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=initial_lr,
        momentum=0.9,
        weight_decay=0.0001
    )

    # Original ResNet learning rate schedule
    milestones = [30, 60]  # Original ResNet milestones
    scheduler = MultiStepLR(  # Fixed MultiStepLR usage
        optimizer,
        milestones=milestones,
        gamma=0.1
    )
    best_loss = float('inf')
    patience_counter = 0
    print("Training settings:")
    print(f"Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Initial LR: {initial_lr}")
    print(f"LR milestones: {milestones}")
    print(f"Epochs: {num_epochs}")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, _, _ in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images = images.to(device)

            # Extract features
            features = []
            x = model.conv1(images)
            x = model.bn1(x)
            x = model.relu(x)
            x = model.maxpool(x)
            x = model.layer1(x)
            features.append(x)
            x = model.layer2(x)
            features.append(x)
            x = model.layer3(x)
            features.append(x)
            x = model.layer4(x)
            features.append(x)
            loss = calculate_feature_loss(features)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss / len(dataloader)
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}, LR: {current_lr:.6f}')
        scheduler.step()
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            patience_counter = 0
            print(f"New best loss: {best_loss:.6f}")
            torch.save(model.state_dict(), MODEL_SAVE_PATH + '.best')
        else:
            patience_counter += 1
            if patience_counter >= 20:  # Increased patience
                print("Early stopping triggered")
                break
    return model

In [38]:
# set variable:continue_training=True if .best model already exist in mydrive.
# y set device to T4 gpu, not CPU.

In [39]:
if __name__ == "__main__":
    # Create dataset with fixed batch size
    dataset = ResNetDataset(DATASET_PATH)
    dataloader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,  # Using fixed batch size
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    print(f"Number of training images: {len(dataset)}")

    # To continue training from saved model:
    trained_model = train_resnet_backbone(
        dataloader,
        continue_training=True  # Set to True to load previous model
    )

    # Save the final model
    print(f"Saving model to {MODEL_SAVE_PATH}")
    torch.save(trained_model.state_dict(), MODEL_SAVE_PATH)
    print("Training completed!")

Number of training images: 5472
Loaded best model from /content/drive/MyDrive/DATASET A MODEL.best
Training settings:
Image size: 224x224
Batch size: 16
Initial LR: 0.075
LR milestones: [30, 60]
Epochs: 200


Epoch 1/200: 100%|██████████| 342/342 [02:19<00:00,  2.46it/s]


Epoch [1/200], Loss: 0.012916, LR: 0.075000
New best loss: 0.012916


Epoch 2/200: 100%|██████████| 342/342 [00:33<00:00, 10.16it/s]


Epoch [2/200], Loss: 0.012541, LR: 0.075000
New best loss: 0.012541


...

Epoch [34/200], Loss: 0.011046, LR: 0.007500


Epoch 35/200: 100%|██████████| 342/342 [00:31<00:00, 10.90it/s]


Epoch [35/200], Loss: 0.011027, LR: 0.007500


Epoch 36/200: 100%|██████████| 342/342 [00:30<00:00, 11.11it/s]


Epoch [36/200], Loss: 0.011004, LR: 0.007500
Early stopping triggered
Saving model to /content/drive/MyDrive/DATASET A MODEL
Training completed!



<br>
###############################<br>
trained_model = train_resnet_backbone(dataloader, continue_training=False)<br>
trained_model = train_resnet_backbone(dataloader,  continue_training=True)<br>
###############################<br>
